In [14]:
#load libraries for data wrangling
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import os
import dask.dataframe as dd
import csv
import importlib


#load function files
import Functions 
from Functions import * 

In [16]:
importlib.reload(Functions)

<module 'Functions' from '/home/livtollanes/10.jan-thesis/Code/Functions.py'>

### Data inspection

In [20]:
data_path = "/home/livtollanes/SocialMarkers"
data = load_data(data_path)

#what does data contain?
data.keys()


dict_keys(['marker_bios', 'marker_followers_bios', 'marker_followers', 'marker_friends', 'readme'])

In [19]:
#Print the first few rows of each DataFrame
for name, df in data.items():
    print(f"First few rows of '{name}':")
    if isinstance(df, str):
        print(df)
    else:
        print(df.head())

First few rows of 'marker_bios':
  twitter_name           id screen_name           name  \
0         Nike  415859364.0        Nike           Nike   
1     adidasFR   25487201.0    adidasFR  adidas France   
2      Lacoste   18481641.0     Lacoste        Lacoste   
3     Converse  183398746.0    Converse       Converse   
4      Moncler  187897893.0     Moncler        Moncler   

                                         description  \
0               #BlackLivesMatter and #StopAsianHate   
1                       𝕭𝖎𝖊𝖓𝖛𝖊𝖓𝖚𝖊 𝖆𝖚 𝕮𝖑𝖚𝖇 𝕺𝖗𝖎𝖌𝖎𝖓𝖆𝖑𝖘.   
2  Moving with the world for 90 years. 🐊 #Lacoste...   
3  Get inspired by the next generation of artists...   
4  Born in the mountains, lives in the city. The ...   

                                      url  timestamp_utc           local_time  \
0                         http://nike.com   1.321655e+09  2011-11-18T22:31:18   
1    http://adidas.fr/impossibleisnothing   1.237546e+09  2009-03-20T10:46:03   
2                 http://laco.st/HLXGd